In [1]:
from datetime import timedelta
from rich import print
from dynaconf import Dynaconf
from arango import ArangoClient
from utinni import Context
from pathlib import Path

from tiro.plugins.utinni import TiroTSPump
from tiro.plugins.arango import ArangoAgent
from tiro.core import Scenario
from influxdb_client import InfluxDBClient

conf = Dynaconf(settings_files=["./config/utinni_config.toml"])
scenario = Scenario.from_yaml(Path("./config/scenario1.yaml"), Path("./config/use1.yaml"))

In [2]:
context = Context(clients=dict(influxdb=InfluxDBClient(**conf.influxdb),
                               arangodb=ArangoAgent(**conf.arangodb)),
                  **conf.on_request)

context.add_pump("tiro", TiroTSPump(scenario=scenario))

In [3]:
table = context.tiro_table(".*Server%(CPU|Memory)Temperature", column="Server", only_ts=False, fill_with_default=True)

In [4]:
context.bind(start=-timedelta(hours=1),
             step=timedelta(minutes=10))

# table.value
# table["MemoryTemperature"].value
table["CPUTemperature"].value

Server,040b2d0c-d675-11ec-b37b-0242ac170002,040b2db6-d675-11ec-b37b-0242ac170002,040b2e2e-d675-11ec-b37b-0242ac170002,040b2e9c-d675-11ec-b37b-0242ac170002,040b2f1e-d675-11ec-b37b-0242ac170002,040b2f8c-d675-11ec-b37b-0242ac170002,040b2ffa-d675-11ec-b37b-0242ac170002,040b3086-d675-11ec-b37b-0242ac170002,040b30f4-d675-11ec-b37b-0242ac170002,040b3180-d675-11ec-b37b-0242ac170002,...,a398ecb8-d6d6-11ec-8d2a-0242ac170002,a398ed26-d6d6-11ec-8d2a-0242ac170002,a398ed94-d6d6-11ec-8d2a-0242ac170002,a398ee20-d6d6-11ec-8d2a-0242ac170002,a398ee84-d6d6-11ec-8d2a-0242ac170002,a398ef1a-d6d6-11ec-8d2a-0242ac170002,a398ef92-d6d6-11ec-8d2a-0242ac170002,a398efec-d6d6-11ec-8d2a-0242ac170002,a398f050-d6d6-11ec-8d2a-0242ac170002,a398f0c8-d6d6-11ec-8d2a-0242ac170002
2022-05-18 17:20:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-18 17:30:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-18 17:40:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-18 17:50:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-18 18:00:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-18 18:10:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-18 18:20:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0


In [5]:
context.tiro_table(pattern=".*Server%(CPU|Memory)Temperature", type="status", as_df=True, include_tags="Server")["CPUTemperature"].value

,Server,value,unit
293,040b5232-d675-11ec-b37b-0242ac170002,20.0,°C
294,040b5296-d675-11ec-b37b-0242ac170002,20.0,°C
295,040b52fa-d675-11ec-b37b-0242ac170002,20.0,°C
296,040b5390-d675-11ec-b37b-0242ac170002,20.0,°C
297,040b53f4-d675-11ec-b37b-0242ac170002,20.0,°C
...,...,...,...
581,a398ef1a-d6d6-11ec-8d2a-0242ac170002,20.0,°C
582,a398ef92-d6d6-11ec-8d2a-0242ac170002,20.0,°C
583,a398efec-d6d6-11ec-8d2a-0242ac170002,20.0,°C
584,a398f050-d6d6-11ec-8d2a-0242ac170002,20.0,°C


In [6]:
print(context.tiro_table(pattern=".*Site", type="status", fill_with_default=True).value)

{
    'Room': {
        '040b2974-d675-11ec-b37b-0242ac170002': {
            'Attribute': {
                'Site': {
                    'value': 'Jackson-Castro',
                    'unit': None,
                    'timestamp': '2022-05-18T06:38:10.356888'
                }
            }
        },
        'a398a3e8-d6d6-11ec-8d2a-0242ac170002': {
            'Attribute': {
                'Site': {
                    'value': 'Myers-Joseph',
                    'unit': None,
                    'timestamp': '2022-05-18T18:16:29.892721'
                }
            }
        }
    }
}